In [12]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import glob

In [13]:
y = ['0','1','2','3','4','5','6','7','8','9','A','a','B','b','C','c','D','d','E','e','F','f','G','g','H','h','I','i','J','j','K','k','L','l','M','m','N','n','O','o','P','p','Q','q','R','r','S','s','T','t','U','u','V','v','W','w','X','x','Y','y','Z','z']; # labels
X = glob.glob("./font/*/*/*.png"); # array of image file paths (features)

In [17]:
def load_and_preprocess_image(file_path):
    # used to read the contents of a file into a tensor. It takes a file path as input and returns a tensor containing the binary data read from the file.
    image = tf.io.read_file(file_path);
    # used to decode an image from a binary string, typically read from a file. It automatically detects the image format and decodes it into a tensor that can be used in TensorFlow.
    image = tf.image.decode_image(image, channels=1); # 1 channel for grayscale, 3 for rgb

    # Preprocess the image (resize, normalize, etc...)
    image = preprocess_image(image);

    return image;

def preprocess_image(image):
    # Explicitly set the shape of the input tensor
    image.set_shape([None, None, 1])
    # Resize
    image = tf.image.resize(image, (224, 224));  # Adjust dimensions as needed

    # Normalized pixel values to the range [0, 1], common practice in image preprocessing for machine learning
    image = image / 255.0;

    return image;

In [18]:
# tensorflow dataset creation
X_tensor = tf.data.Dataset.from_tensor_slices(X);
# applies the function to each element of the dataset X_tensor.
X_tensor = X_tensor.map(load_and_preprocess_image);